# <span style="color:gray">ipyrad-analysis toolkit:</span> mrBayes

Bayesian phylogenetic inference can provide several advantages over ML approaches, particularly with regards to inferring dated trees (separating rate and time), and because they assess support differently, using a posterior sample of trees inferred from the full data set as opposed to bootstrap resampling of the data set. This may be particularly relevant to RAD-seq data that contains missing data. 

Bayesian inference programs often contain waaay too many options, which make them difficult to use. But it's kind of necessary in order to make informed decisions when setting priors on parameters, as opposed to treating the analysis like a black box. That being said, I've written a sort of blackbox tool for running mrbayes analyses. Once you've established a set of parameters that are best for your analysis this tool is useful to then automate it across many loci (e.g., see [`ipa.treeslider()`](https://ipyrad.readthedocs.io/en/latest/API-analysis/cookbook-treeslider.html)). 

### Required software

In [1]:
# conda install ipyrad -c conda-forge -c bioconda
# conda install toytree -c conda-forge
# conda install mrbayes -c conda-forge -c bioconda

In [7]:
import ipyrad.analysis as ipa
import toytree

In [8]:
# check mrbayes version
! mb -v

MrBayes, Bayesian Analysis of Phylogeny

Version:   3.2.7
Features:  SSE AVX FMA Beagle readline
Host type: x86_64-conda-linux-gnu (CPU: x86_64)
Compiler:  gnu 7.5.0


### Sequence database file

In [9]:
# the seqs.hdf5 file produced by ipyrad
SEQSFILE = "/tmp/oaks.seqs.hdf5"

In [10]:
# download example seqs file if not already present (~500Mb, takes ~5 minutes)
URL = "https://www.dropbox.com/s/c1u89nwuuv8e6ie/virentes_ref.seqs.hdf5?raw=1"
ipa.download(URL, path=SEQSFILE);

file already exists


### Write a nexus alignment (using window extracter)
This tool will extract RAD loci mapping to the first scaffold and with no missing data across the selected 10 samples and write the sequence data to a nexus file. See the window_extracter docs for more details.

In [21]:
# samples in include
IMAP = {
    'include': [
        "LALC2", "FLSF47", "FLCK18", "CUSV6",
        "CRL0030", "MXED8",  "BJSB3", "AR"
    ],
}

# init wex tool to select data
wex = ipa.window_extracter(
    data=SEQSFILE,
    name="oaks-chr1-aln",
    workdir="/tmp",
    scaffold_idxs=0,
    mincov=1.0,
    imap=IMAP,
)

# show stats for this alignment
display(wex.stats)

# write the alignment file
wex.run(nexus=True, force=True)

,scaffold,start,end,sites,snps,missing,samples
prefilter,Qrob_Chr01,0,None,890747,6133,0.46,8
postfilter,Qrob_Chr01,0,None,219136,2525,0.00,8


Wrote data to /tmp/oaks-chr1-aln.nex


### Setting mb param settings

The purpose of the ipyrad wrapper tool for mrbayes is merely for convenience. You should still take care to understand the priors and parameters that you are selecting by reading the mrbayes documentation. We only support two relatively simple models, a clock and non-clock model with a number of parameters listed in the `.params` attribute of the analysis object. Here I demonstrate the clock model with uncorrelated lognormal distributed rate variation and a birth-death tree prior. This run, which samples 5M iterations takes about 20 hours on 4 cores. 

In [22]:
# the path to your NEXUS formatted file
NEXFILE = "/tmp/oaks-chr1-aln.nex"

In [25]:
# init mrbayes object with input data and (optional) parameter options
mb = ipa.mrbayes(
    name="oaks-chr1-mb",
    data=NEXFILE,
    clock_model=2,
    ngen=int(5e6),
    samplefreq=int(5e3),
    nruns=2,
    constraints=
)

In [27]:
# modify a param and show all params
mb.params.nruns = 1
mb.params

brlenspr      clock:uniform       
clockratepr   normal(0.01,0.005)  
clockvarpr    igr                 
igrvarpr      exp(10.0)           
nchains       4                   
ngen          5000000             
nruns         1                   
samplefreq    5000                
topologypr    uniform             

In [28]:
# print the mb nexus string; this can be modified by changing .params
print(mb.nexus_string)

#NEXUS

[log block]
log start filename=/home/deren/Documents/ipyrad/testdocs/analysis/analysis-mb/oaks-chr1-mb.nex.log replace;

[data block]
execute /tmp/oaks-chr1-aln.nex;

[tree block]


[mb block]
begin mrbayes;
  set autoclose=yes nowarn=yes;

  lset nst=6 rates=gamma;

  prset brlenspr=clock:uniform;
  prset clockvarpr=igr;
  prset igrvarpr=exp(10.0);
  prset clockratepr=normal(0.01,0.005);
  prset topologypr=uniform;

  

  mcmcp ngen=5000000 nrun=1 nchains=4;
  mcmcp relburnin=yes burninfrac=0.25;
  mcmcp samplefreq=5000;
  mcmcp printfreq=10000 diagnfr=5000;
  mcmcp filename=/home/deren/Documents/ipyrad/testdocs/analysis/analysis-mb/oaks-chr1-mb.nex;
  mcmc;

  sump filename=/home/deren/Documents/ipyrad/testdocs/analysis/analysis-mb/oaks-chr1-mb.nex;
  sumt filename=/home/deren/Documents/ipyrad/testdocs/analysis/analysis-mb/oaks-chr1-mb.nex contype=allcompat;
end;

[log block]
log stop filename=/home/deren/Documents/ipyrad/testdocs/analysis/analysis-mb/oaks-chr1-mb.nex.log app

### Run mrbayes

This takes about 20 minutes to run on my laptop for a data set with 13 samples and ~1.2M SNPs and about 14% missing data. For a publication quality analyses you will likely want to run it quite a bit longer and to test for convergence of your runs using a tool like `tracer` and `treeannotator`.

In [8]:
# run the command, (options: block until finishes; overwrite existing)
mb.run(block=True, force=True)

job pedicularis-min10-5M finished successfully


### Accessing results

You can access the results files of the mrbayes analysis from the mb analysis object from the `.trees` attribute, or of course you can also write out the full path to the result files which will be located in your working directory which can be set as a parameter option, otherwise is defaulted to `"./analysis-mb/".`

In [9]:
# you can access the tree results from the mb analysis object
mb.trees

constre   ~/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10-5M.nex.con.tre
info      ~/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10-5M.nex.lstat
posttre   ~/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10-5M.nex.t

In [10]:
# for example to select the consensus tree path
mb.trees.constre

'/home/deren/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10-5M.nex.con.tre'

### Check the parameter values and convergence

On this very large data set (in terms of number of sites) we can see that the posterior has not explored the parameter space very well by the end of this run. We would hope to see the ESS score for all parameters above 100. Here the TH, TL, net_speciation, and clockrate parameters have low convergence scores. 

In [12]:
# show the convergence statistics (from the .pstat mb output file)
mb.convergence_stats.round(3)

,Mean,Variance,Lower,Upper,Median,ESS
Parameter,,,,,,
TH,0.055,0.000,0.045,0.067,0.054,4.014
TL,0.443,0.005,0.328,0.564,0.438,4.031
r(A<->C),0.102,0.000,0.100,0.105,0.102,325.813
r(A<->G),0.286,0.000,0.283,0.289,0.286,267.213
r(A<->T),0.112,0.000,0.109,0.114,0.112,341.615
r(C<->G),0.102,0.000,0.100,0.105,0.102,376.000
r(C<->T),0.296,0.000,0.293,0.299,0.296,376.000
r(G<->T),0.102,0.000,0.100,0.104,0.102,376.000
pi(A),0.297,0.000,0.296,0.298,0.297,238.441


### Plotting mb consesus tree
You can plot either the consensus tree (`.nex.cons.tre`) or the posterior distribution of trees (`.nex.t`) produced by a mrbayes run by loading the single tree or list of trees with `toytree` as demonstrated below. 

In [16]:
# load from the .trees attribute or from the saved tree file
tre = toytree.tree(mb.trees.constre, tree_format=10)

# draw the tree with styling
tre.draw(
    tip_labels_align=True, 
    scalebar=True,
    #node_labels="support",
);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="te14a3fdfdbf14cf0bdaf7a448b2df50c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 415.0 275.0" width="415.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 39618_rex_SRR1754723 38362_rex_SRR1754725 35236_rex_SRR1754731 40578_rex_SRR1754724 35855_rex_SRR1754726 30556_thamno_SRR1754720 33413_thamno_SRR1754728 41954_cyathophylloides_SRR1754721 41478_cyathophylloides_SRR1754722 30686_cyathophylla_SRR1754730 29154_superba_SRR1754715 33588_przewalskii_SRR1754727 32082_przewalskii_SRR1754729 0 6 12 18

### Plotting mb posterior distribution of trees

In [27]:
# load from the .trees attribute or from the saved tree file
mtre = toytree.mtree(mb.trees.posttre)

# remove the burnin manually
mtre.treelist = mtre.treelist[10:]

# draw a few trees on a shared axis
mtre.draw_tree_grid(
    nrows=1, ncols=4, start=20, shared_axis=True,
    height=400, 
);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t43d4d3e4c0024277b045c4d1eda1c3b1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 400.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 38362_rex_SRR1754725 39618_rex_SRR1754723 35236_rex_SRR1754731 40578_rex_SRR1754724 35855_rex_SRR1754726 30556_thamno_SRR1754720 33413_thamno_SRR1754728 41478_cyathophylloides_SRR1754722 41954_cyathophylloides_SRR1754721 29154_superba_SRR1754715 30686_cyathophylla_SRR1754730 32082_przewalskii_SRR1754729 33588_przewalskii_SRR1754727 38362_rex_SRR1754725 39618_rex_SRR1754723 35236_rex_SRR1754731 35855_rex_SRR1754726 40578_rex_SRR1754724 30556_thamno_SRR1754720 33413_thamno_SRR1754728 29154_superba_SRR1754715 30686_cyathophylla_SRR1754730 41478_cyathophylloides_SRR1754722 41954_cyathophylloides_SRR1754721 33588_przewalskii_SRR1754727 32082_przewalskii_SRR1754729 38362_rex_SRR1754725 39618_rex_SRR1754723 35236_rex_SRR1754731 40578_rex_SRR1754724 35855_rex_SRR1754726 30556_thamno_SRR1754720 33413_thamno_SRR1754728 30686_cyathophylla_SRR1754730 29154_superba_SRR1754715 41954_cyathophylloides_SRR1754721 41478_cyathophylloides_SRR1754722 33588_przewalskii_SRR1754727 32082_przewalskii_SRR1754729 38362_rex_SRR1754725 39618_rex_SRR1754723 35236_rex_SRR1754731 35855_rex_SRR1754726 40578_rex_SRR1754724 30556_thamno_SRR1754720 33413_thamno_SRR1754728 29154_superba_SRR1754715 30686_cyathophylla_SRR1754730 41478_cyathophylloides_SRR1754722 41954_cyathophylloides_SRR1754721 33588_przewalskii_SRR1754727 32082_przewalskii_SRR1754729 0.00 0.02 0.03

The command below plots a cloud tree diagram across the ~90 trees in the posterior distribution. Because we have no fossil calibrations for this data the root age is not of great interest so I use the `.mod` fuction below to scale the root height of each tree to 1.0. This will highlight variation among trees in relative node heights. Below you can see that in the posterior set of trees there some variation around node heights in the middle of the tree, and there does not appear to be any variation in the topology recovered, which is not uncommon for large concatenated data sets. 

In [38]:
# set root height of all trees to 1.0
mtre.treelist = [i.mod.node_scale_root_height(1.0) for i in mtre.treelist]

In [39]:
# draw the posterior of trees overlapping
mtre.draw_cloud_tree(
    width=400, height=300,
    html=True,
    edge_colors="red",
    edge_style={"stroke-opacity": 0.02},
    tip_labels={i: i.rsplit("_", 1)[0] for i in mtre.treelist[0].get_tip_labels()},
);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t7f77d0cf0f154e41ae4def4edc1daac4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 91.46207293020927 139.49538774233895 L 103.1613615155384 110

### Save figures

In [14]:
import toyplot.pdf
canvas, axes = mtre.draw_cloud_tree(
    width=400, height=300,
    html=True,
    edge_colors="red",
    tip_labels={i: i.rsplit("_", 1)[0] for i in mtre.treelist[0].get_tip_labels()}
);
toyplot.pdf.render(canvas, "./pedicularis-min10-5M-mb-tree.pdf")

### What else?

If you have reference mapped data then you should see the `.treeslider()` tool to infer trees in sliding windows along scaffolds; or the `.extractor()` tool to extract, filter, and concatenate RAD loci within a given window (e.g., near some known gene).